# Introduction to Keras callbacks

> In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. This is the summary of lecture "Custom Models, Layers and Loss functions with Tensorflow" from DeepLearning.AI.

- toc: true 
- badges: true
- comments: true
- author: Chanseok Kang
- categories: [Python, Coursera, Tensorflow, DeepLearining.AI]
- image: 

## TL;DR - Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Packages

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pyplot as plt
import io
from PIL import Image
import numpy as np
import pandas as pd
import datetime
import math

tf.get_logger().setLevel('INFO')

In [2]:
print('Tensorflow version: {}'.format(tf.__version__))

Tensorflow version: 2.4.1


## Examples of Keras callback applications

The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [3]:
# horses_or_humans datasets
splits, info = tfds.load('horses_or_humans', try_gcs=True, as_supervised=True, with_info=True, split=['train[:80%]', 'test[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

In [4]:
num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes
print(num_examples, num_classes)

1027 2


In [5]:
SIZE = 150 #@param {type:'slider', min:64, max=300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [6]:
def format_image(image, label):
    image = tf.image.resize(image, IMAGE_SIZE) / 255.0
    return image, label

In [7]:
BATCH_SIZE=32 #@param {type:"integer"}

In [8]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [9]:
for image_batch, label_batch in train_batches.take(1):
    pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [10]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(dense_units, activation='relu'),
        tf.keras.layers.Dense(2, activation='softmax')
    ])
    return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [11]:
model = build_model(dense_units=256)
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [12]:
logdir = os.path.join('logs', datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, epochs=10, validation_data=validation_batches, callbacks=[tensorboard_callback])

Epoch 1/10
26/26 [==============================] - 2s 32ms/step - loss: 0.6811 - accuracy: 0.5546 - val_loss: 0.6969 - val_accuracy: 0.4902
Epoch 2/10
26/26 [==============================] - 1s 16ms/step - loss: 0.6467 - accuracy: 0.6274 - val_loss: 0.6752 - val_accuracy: 0.5490
Epoch 3/10
26/26 [==============================] - 1s 16ms/step - loss: 0.6063 - accuracy: 0.7291 - val_loss: 0.7409 - val_accuracy: 0.4902
Epoch 4/10
26/26 [==============================] - 1s 16ms/step - loss: 0.5578 - accuracy: 0.7614 - val_loss: 0.8267 - val_accuracy: 0.4902
Epoch 5/10
26/26 [==============================] - 1s 16ms/step - loss: 0.5169 - accuracy: 0.7451 - val_loss: 0.7518 - val_accuracy: 0.4902
Epoch 6/10
26/26 [==============================] - 1s 16ms/step - loss: 0.4436 - accuracy: 0.8081 - val_loss: 0.5313 - val_accuracy: 0.7059
Epoch 7/10
26/26 [==============================] - 1s 17ms/step - loss: 0.4178 - accuracy: 0.8276 - val_loss: 0.7424 - val_accuracy: 0.5294
Epoch 8/10
26

In [20]:
%tensorboard --logdir logs --port=8008

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [21]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5
26/26 - 1s - loss: 0.6795 - accuracy: 0.5669 - val_loss: 0.6799 - val_accuracy: 0.4902

Epoch 00001: saving model to weights.01-0.68.h5
Epoch 2/5
26/26 - 1s - loss: 0.6227 - accuracy: 0.6642 - val_loss: 0.6620 - val_accuracy: 0.5098

Epoch 00002: saving model to weights.02-0.66.h5
Epoch 3/5
26/26 - 1s - loss: 0.5450 - accuracy: 0.7470 - val_loss: 0.5719 - val_accuracy: 0.6667

Epoch 00003: saving model to weights.03-0.57.h5
Epoch 4/5
26/26 - 1s - loss: 0.5033 - accuracy: 0.7689 - val_loss: 0.9673 - val_accuracy: 0.5098

Epoch 00004: saving model to weights.04-0.97.h5
Epoch 5/5
26/26 - 1s - loss: 0.4358 - accuracy: 0.8163 - val_loss: 0.9052 - val_accuracy: 0.5294

Epoch 00005: saving model to weights.05-0.91.h5


In [22]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])

26/26 - 1s - loss: 0.6766 - accuracy: 0.5791 - val_loss: 0.6781 - val_accuracy: 0.5686

Epoch 00001: saving model to saved_model
INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


In [23]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

Epoch 1/2
26/26 - 1s - loss: 0.6784 - accuracy: 0.5450 - val_loss: 0.6521 - val_accuracy: 0.8039

Epoch 00001: saving model to model.h5
Epoch 2/2
26/26 - 1s - loss: 0.6285 - accuracy: 0.6691 - val_loss: 0.6768 - val_accuracy: 0.5294

Epoch 00002: saving model to model.h5


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [24]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 1s - loss: 0.6697 - accuracy: 0.6083 - val_loss: 0.7541 - val_accuracy: 0.4902
Epoch 2/50
26/26 - 1s - loss: 0.6004 - accuracy: 0.7348 - val_loss: 0.7659 - val_accuracy: 0.5098
Epoch 3/50
26/26 - 1s - loss: 0.5476 - accuracy: 0.7603 - val_loss: 0.7079 - val_accuracy: 0.5294
Epoch 4/50
26/26 - 1s - loss: 0.4586 - accuracy: 0.8127 - val_loss: 0.5082 - val_accuracy: 0.8431
Epoch 5/50
26/26 - 1s - loss: 0.3939 - accuracy: 0.8406 - val_loss: 0.9250 - val_accuracy: 0.5490
Epoch 6/50
26/26 - 1s - loss: 0.3358 - accuracy: 0.8735 - val_loss: 0.9624 - val_accuracy: 0.5686
Epoch 7/50
26/26 - 1s - loss: 0.2686 - accuracy: 0.9246 - val_loss: 0.7375 - val_accuracy: 0.7255
Restoring model weights from the end of the best epoch.
Epoch 00007: early stopping


## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [25]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 1s 19ms/step - loss: 0.7008 - accuracy: 0.5189 - val_loss: 0.7107 - val_accuracy: 0.4902
Epoch 2/5
26/26 [==============================] - 1s 16ms/step - loss: 0.6378 - accuracy: 0.6557 - val_loss: 0.6065 - val_accuracy: 0.5294
Epoch 3/5
26/26 [==============================] - 1s 16ms/step - loss: 0.5807 - accuracy: 0.7298 - val_loss: 0.5507 - val_accuracy: 0.7059
Epoch 4/5
26/26 [==============================] - 1s 16ms/step - loss: 0.5681 - accuracy: 0.7191 - val_loss: 0.6519 - val_accuracy: 0.6078
Epoch 5/5
26/26 [==============================] - 1s 16ms/step - loss: 0.5316 - accuracy: 0.7255 - val_loss: 0.6350 - val_accuracy: 0.6471


In [26]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.588808,0.679656,0.490196,0.710690
1,1,0.678832,0.618920,0.529412,0.606464
2,2,0.711679,0.584460,0.705882,0.550742
3,3,0.763990,0.527732,0.607843,0.651865
4,4,0.774939,0.494997,0.647059,0.635014


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [27]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])

Epoch 1/5

Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
26/26 [==============================] - 1s 25ms/step - loss: 0.6934 - accuracy: 0.5258 - val_loss: 0.6919 - val_accuracy: 0.4902
Epoch 2/5

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
26/26 [==============================] - 1s 19ms/step - loss: 0.6702 - accuracy: 0.5562 - val_loss: 0.6906 - val_accuracy: 0.4902
Epoch 3/5

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
26/26 [==============================] - 1s 16ms/step - loss: 0.6594 - accuracy: 0.5824 - val_loss: 0.6807 - val_accuracy: 0.4902
Epoch 4/5

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
26/26 [==============================] - 1s 16ms/step - loss: 0.6599 - accuracy: 0.6085 - val_loss: 0.6808 - val_accuracy: 0.4902
Epoch 5/5

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0003125.
26/26 [==============================] - 1s 16ms/step - loss: 0.6560 - accu

In [28]:
%tensorboard --logdir log_dir

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [29]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

Epoch 1/50
26/26 [==============================] - 1s 22ms/step - loss: 0.6831 - accuracy: 0.5864 - val_loss: 0.6870 - val_accuracy: 0.4902
Epoch 2/50
26/26 [==============================] - 1s 17ms/step - loss: 0.6416 - accuracy: 0.6773 - val_loss: 0.6623 - val_accuracy: 0.8039
Epoch 3/50
26/26 [==============================] - 1s 16ms/step - loss: 0.5877 - accuracy: 0.7788 - val_loss: 0.9273 - val_accuracy: 0.4902

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0019999999552965165.
Epoch 4/50
26/26 [==============================] - 1s 16ms/step - loss: 0.5352 - accuracy: 0.7535 - val_loss: 0.6460 - val_accuracy: 0.5686
Epoch 5/50
26/26 [==============================] - 1s 17ms/step - loss: 0.5079 - accuracy: 0.8116 - val_loss: 0.7629 - val_accuracy: 0.4902

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.001.
Epoch 6/50
26/26 [==============================] - 1s 16ms/step - loss: 0.4893 - accuracy: 0.8444 - val_loss: 0.7508 - val_accuracy: 0.4902

Epoch 0

In [30]:
%tensorboard --logdir log_dir

Reusing TensorBoard on port 6008 (pid 74529), started 0:00:43 ago. (Use '!kill 74529' to kill it.)